In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("../../../Downloads/data.csv")

In [3]:
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:,0:1024]
Y = Y[:,1024]

X_train = X[:70000,:]
X_train = X_train/255.
X_test = X[70000:72001,:]
X_test = X_test/255.


#Reshape
Y = Y.reshape(Y.shape[0],1)
y_train = Y[0:70000,:]
y_train = y_train.T
y_test = Y[70000:72001,:]
y_test = y_test.T

In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(70000, 1024)
(2000, 1024)
(1, 70000)
(1, 2000)


In [5]:
image_x = 32
image_y = 32

train_y = np_utils.to_categorical(y_train)
test_y = np_utils.to_categorical(y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test = X_test.reshape(X_test.shape[0],image_x,image_y,1)

In [6]:
print(train_y.shape)

(70000, 37)


In [17]:
def keras_model(image_x,image_y):
    num_of_classes = 37
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size=(5,5), input_shape=(image_x,image_y,1),activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(Conv2D(64,(5,5),activation ='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss ='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath= 'devanagari.h5'
    checkpoint1 = ModelCheckpoint(filepath,monitor='val_acc',verbose=1, save_best_only=True,mode='max')
    callbacks_list = [checkpoint1]
    
    return model,callbacks_list

In [19]:
model, callbacks_list= keras_model(image_x,image_y)
model.fit(X_train,train_y, validation_data=(X_test,test_y),epochs=2,batch_size = 64,callbacks=callbacks_list)
scores = model.evaluate(X_test,test_y,verbose=0)
print("CNN Error: %.2f%%", (100-scores[1]*100))
print_summary(model)
model.save('devanagari.h5')

Train on 70000 samples, validate on 2000 samples
Epoch 1/2
70000/70000 [==============================] - 536s 8ms/step - loss: 0.8419 - acc: 0.7576 - val_loss: 0.3492 - val_acc: 0.8955

Epoch 00001: val_acc improved from -inf to 0.89550, saving model to devanagari.h5
Epoch 2/2
70000/70000 [==============================] - 590s 8ms/step - loss: 0.2561 - acc: 0.9224 - val_loss: 0.2239 - val_acc: 0.9275

Epoch 00002: val_acc improved from 0.89550 to 0.92750, saving model to devanagari.h5
CNN Error: %.2f%% 7.25
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        51264     
_______________________